In [ ]:
"""
Command-Line Contact Book Application
A simple contact management system that stores data in a text file.
"""

import os
import json
from datetime import datetime
from typing import List, Dict, Optional

class Contact:
    """Represents a single contact."""
    
    def __init__(self, name: str, phone: str, email: str = "", address: str = "", notes: str = ""):
        self.name = name
        self.phone = phone
        self.email = email
        self.address = address
        self.notes = notes
        self.created_at = datetime.now().isoformat()
    
    def to_dict(self) -> Dict:
        """Convert contact to dictionary."""
        return {
            'name': self.name,
            'phone': self.phone,
            'email': self.email,
            'address': self.address,
            'notes': self.notes,
            'created_at': self.created_at
        }
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'Contact':
        """Create contact from dictionary."""
        contact = cls(
            data['name'],
            data['phone'],
            data.get('email', ''),
            data.get('address', ''),
            data.get('notes', '')
        )
        contact.created_at = data.get('created_at', datetime.now().isoformat())
        return contact
    
    def __str__(self) -> str:
        """String representation of contact."""
        return f"""
{'='*50}
Name:       {self.name}
Phone:      {self.phone}
Email:      {self.email if self.email else 'N/A'}
Address:    {self.address if self.address else 'N/A'}
Notes:      {self.notes if self.notes else 'N/A'}
Added:      {self.created_at[:10]}
{'='*50}"""




In [5]:
class ContactBook:
    """Manages the contact book operations."""
    
    def __init__(self, filename: str = "contacts.txt"):
        self.filename = filename
        self.contacts: List[Contact] = []
        self.load_contacts()
    
    def load_contacts(self):
        """Load contacts from file."""
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r') as f:
                    data = f.read()
                    if data:
                        contacts_data = json.loads(data)
                        self.contacts = [Contact.from_dict(c) for c in contacts_data]
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Warning: Could not load contacts properly. Starting fresh. Error: {e}")
                self.contacts = []
    
    def save_contacts(self):
        """Save contacts to file."""
        contacts_data = [c.to_dict() for c in self.contacts]
        with open(self.filename, 'w') as f:
            json.dump(contacts_data, f, indent=2)
    
    def add_contact(self, contact: Contact) -> bool:
        """Add a new contact."""
        # Check for duplicate phone number
        if any(c.phone == contact.phone for c in self.contacts):
            return False
        self.contacts.append(contact)
        self.save_contacts()
        return True
    
    def view_all_contacts(self) -> List[Contact]:
        """Return all contacts sorted by name."""
        return sorted(self.contacts, key=lambda x: x.name.lower())
    
    def search_contacts(self, query: str) -> List[Contact]:
        """Search contacts by name, phone, or email."""
        query = query.lower()
        results = []
        for contact in self.contacts:
            if (query in contact.name.lower() or 
                query in contact.phone or 
                query in contact.email.lower()):
                results.append(contact)
        return sorted(results, key=lambda x: x.name.lower())
    
    def delete_contact(self, phone: str) -> bool:
        """Delete a contact by phone number."""
        for i, contact in enumerate(self.contacts):
            if contact.phone == phone:
                del self.contacts[i]
                self.save_contacts()
                return True
        return False
    
    def update_contact(self, phone: str, updated_contact: Contact) -> bool:
        """Update an existing contact."""
        for i, contact in enumerate(self.contacts):
            if contact.phone == phone:
                self.contacts[i] = updated_contact
                self.save_contacts()
                return True
        return False
    
    def get_contact_by_phone(self, phone: str) -> Optional[Contact]:
        """Get a specific contact by phone number."""
        for contact in self.contacts:
            if contact.phone == phone:
                return contact
        return None




In [6]:
class ContactBookCLI:
    """Command-line interface for the contact book."""
    
    def __init__(self):
        self.contact_book = ContactBook()
    
    def display_menu(self):
        """Display the main menu."""
        print("\n" + "="*60)
        print(" "*20 + "CONTACT BOOK MENU")
        print("="*60)
        print("1. Add New Contact")
        print("2. View All Contacts")
        print("3. Search Contacts")
        print("4. Update Contact")
        print("5. Delete Contact")
        print("6. Export Contacts")
        print("7. Statistics")
        print("8. Exit")
        print("="*60)
    
    def get_contact_details(self, existing_contact: Optional[Contact] = None) -> Contact:
        """Get contact details from user input."""
        print("\nEnter contact details:")
        
        name = input("Name (required): ").strip()
        while not name:
            print("Name cannot be empty!")
            name = input("Name (required): ").strip()
        
        if existing_contact:
            phone = input(f"Phone (current: {existing_contact.phone}): ").strip()
            if not phone:
                phone = existing_contact.phone
        else:
            phone = input("Phone (required): ").strip()
            while not phone:
                print("Phone cannot be empty!")
                phone = input("Phone (required): ").strip()
        
        email = input("Email (optional): ").strip()
        address = input("Address (optional): ").strip()
        notes = input("Notes (optional): ").strip()
        
        return Contact(name, phone, email, address, notes)
    
    def add_contact(self):
        """Add a new contact."""
        print("\n--- ADD NEW CONTACT ---")
        contact = self.get_contact_details()
        
        if self.contact_book.add_contact(contact):
            print(f"\n✓ Contact '{contact.name}' added successfully!")
        else:
            print(f"\n✗ A contact with phone number {contact.phone} already exists!")
    
    def view_all_contacts(self):
        """View all contacts."""
        print("\n--- ALL CONTACTS ---")
        contacts = self.contact_book.view_all_contacts()
        
        if not contacts:
            print("\nNo contacts found in the contact book.")
        else:
            print(f"\nTotal contacts: {len(contacts)}")
            for contact in contacts:
                print(contact)
    
    def search_contacts(self):
        """Search for contacts."""
        print("\n--- SEARCH CONTACTS ---")
        query = input("Enter search term (name/phone/email): ").strip()
        
        if not query:
            print("Search term cannot be empty!")
            return
        
        results = self.contact_book.search_contacts(query)
        
        if not results:
            print(f"\nNo contacts found matching '{query}'")
        else:
            print(f"\nFound {len(results)} contact(s) matching '{query}':")
            for contact in results:
                print(contact)
    
    def update_contact(self):
        """Update an existing contact."""
        print("\n--- UPDATE CONTACT ---")
        phone = input("Enter phone number of contact to update: ").strip()
        
        contact = self.contact_book.get_contact_by_phone(phone)
        if not contact:
            print(f"\nNo contact found with phone number {phone}")
            return
        
        print(f"\nCurrent details for {contact.name}:")
        print(contact)
        
        print("\nEnter new details (press Enter to keep current value):")
        updated_contact = self.get_contact_details(contact)
        
        if self.contact_book.update_contact(phone, updated_contact):
            print(f"\n✓ Contact updated successfully!")
        else:
            print(f"\n✗ Failed to update contact.")
    
    def delete_contact(self):
        """Delete a contact."""
        print("\n--- DELETE CONTACT ---")
        phone = input("Enter phone number of contact to delete: ").strip()
        
        contact = self.contact_book.get_contact_by_phone(phone)
        if not contact:
            print(f"\nNo contact found with phone number {phone}")
            return
        
        print(f"\nContact to be deleted:")
        print(contact)
        
        confirm = input("\nAre you sure you want to delete this contact? (y/n): ").lower()
        if confirm == 'y':
            if self.contact_book.delete_contact(phone):
                print(f"\n✓ Contact deleted successfully!")
            else:
                print(f"\n✗ Failed to delete contact.")
        else:
            print("\nDeletion cancelled.")
    
    def export_contacts(self):
        """Export contacts to a readable text file."""
        print("\n--- EXPORT CONTACTS ---")
        filename = input("Enter export filename (default: contacts_export.txt): ").strip()
        if not filename:
            filename = "contacts_export.txt"
        
        contacts = self.contact_book.view_all_contacts()
        
        try:
            with open(filename, 'w') as f:
                f.write("CONTACT BOOK EXPORT\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Total Contacts: {len(contacts)}\n")
                f.write("="*60 + "\n\n")
                
                for contact in contacts:
                    f.write(str(contact) + "\n")
            
            print(f"\n✓ Contacts exported successfully to {filename}")
        except Exception as e:
            print(f"\n✗ Failed to export contacts: {e}")
    
    def show_statistics(self):
        """Show statistics about the contact book."""
        print("\n--- CONTACT BOOK STATISTICS ---")
        contacts = self.contact_book.contacts
        
        if not contacts:
            print("\nNo contacts to analyze.")
            return
        
        total = len(contacts)
        with_email = sum(1 for c in contacts if c.email)
        with_address = sum(1 for c in contacts if c.address)
        with_notes = sum(1 for c in contacts if c.notes)
        
        print(f"\nTotal Contacts: {total}")
        print(f"Contacts with Email: {with_email} ({with_email*100//total}%)")
        print(f"Contacts with Address: {with_address} ({with_address*100//total}%)")
        print(f"Contacts with Notes: {with_notes} ({with_notes*100//total}%)")
        
        # Find contact added most recently
        most_recent = max(contacts, key=lambda x: x.created_at)
        print(f"\nMost Recently Added: {most_recent.name} ({most_recent.created_at[:10]})")
    
    def run(self):
        """Main application loop."""
        print("\n" + "="*60)
        print(" "*15 + "WELCOME TO CONTACT BOOK")
        print("="*60)
        
        while True:
            self.display_menu()
            choice = input("\nEnter your choice (1-8): ").strip()
            
            if choice == '1':
                self.add_contact()
            elif choice == '2':
                self.view_all_contacts()
            elif choice == '3':
                self.search_contacts()
            elif choice == '4':
                self.update_contact()
            elif choice == '5':
                self.delete_contact()
            elif choice == '6':
                self.export_contacts()
            elif choice == '7':
                self.show_statistics()
            elif choice == '8':
                print("\nThank you for using Contact Book. Goodbye!")
                break
            else:
                print("\n✗ Invalid choice! Please enter a number between 1 and 8.")
            
            input("\nPress Enter to continue...")


def main():
    """Entry point of the application."""
    try:
        app = ContactBookCLI()
        app.run()
    except KeyboardInterrupt:
        print("\n\nApplication interrupted. Goodbye!")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


if __name__ == "__main__":
    main()


               WELCOME TO CONTACT BOOK

                    CONTACT BOOK MENU
1. Add New Contact
2. View All Contacts
3. Search Contacts
4. Update Contact
5. Delete Contact
6. Export Contacts
7. Statistics
8. Exit



--- CONTACT BOOK STATISTICS ---

Total Contacts: 3
Contacts with Email: 3 (100%)
Contacts with Address: 3 (100%)
Contacts with Notes: 0 (0%)

Most Recently Added: Clark (2025-08-19)

                    CONTACT BOOK MENU
1. Add New Contact
2. View All Contacts
3. Search Contacts
4. Update Contact
5. Delete Contact
6. Export Contacts
7. Statistics
8. Exit

Thank you for using Contact Book. Goodbye!
